# 8.4 - Mortgages, an Extended Example

Build a program that examines the costs of three kinds of loans:
1. A fixed-rate mortgage with no points
2. A fixed-rate mortgage with points
3. A mortgage with an initial teaser rate forllowed by a higher rate for the duration.

In [5]:
'''Code will be structured to include a Mortgage class, and subclasses corresponding to each of the three kinds of mortgages.'''
'''The Mortgage class will contain methods that are shared by each of its subclasses, but it is not intended to be instantiated \
directly.'''
'''That is, no objects of type Mortgage will be created.'''

'''The function findPayment computes the size of the fixed monthly payment needed to pay off the loan, including interest, by the \
end of its term.'''

'''For __init__: all Mortgage instance variables: initial loan amount, the monthly interest rate, the duration of the loan in \
months, payment list, etc.'''

def findPayment(loan, r, m):
    '''Assumes: loan and r are floats, m an int'''
    '''Returns the monthly payment for a mortgage of size loan at a monthly rate of r for m months'''
    return loan*((r*(1+r)**m) / ((1+r)**m - 1))

class Mortgage(object):
    '''Abstract class for building different kinds of mortgages'''
    def __init__(self, loan, annRate, months):
        '''Assumes: loan and annRate are floats, months an int'''
        '''Creates a new mortgage of size loan, duration months, and annual annual rate annRate'''
        self.loan = loan
        self.rate = annRate/12
        self.months = months
        self.paid = [0.0]
        self.outstanding = [loan]
        self.payment = findPayment(loan, self.rate, months)
        self.legend = None
        
    def makePayment(self):
        '''Make a payment'''
        self.paid.append(self.payment)
        reduction = self.payment - self.outstanding[-1]*self.rate
        self.outstanding.append(self.outstanding[-1] - reduction)
        
    def getTotalPaid(self):
        '''Return the total amount paid so far'''
        return sum(self.paid)
    
    def __str__(self):
        return self.legend
    
class Fixed(Mortgage):
    def __init__(self, loan, r, months):
        Mortgage.__init__(self, loan, r, months)
        self.legend = 'Fixed, ' + str(round(r*100, 2)) + '%'

class FixedWithPts(Mortgage):
    def __init__(self, loan, r, months, pts):
        Mortgage.__init__(self, loan, r, months)
        self.pts = pts
        self.paid = [loan*(pts/100)]
        self.legend = 'Fixed, ' + str(round(r*100, 2)) + '%, ' + str(pts) + ' points'

class TwoRate(Mortgage):
    def __init__(self, loan, r, months, teaserRate, teaserMonths):
        Mortgage.__init__(self, loan, teaserRate, months)
        self.teaserMonths = teaserMonths
        self.teaserRate = teaserRate
        self.nextRate = r/12
        self.legend = str(teaserRate*100) + '% for ' + str(self.teaserMonths) + ' months, then ' + str(round(r*100, 2)) + '%'

    def makePayment(self):
        if len(self.paid) == self.teaserMonths + 1:
            self.rate = self.nextRate
            self.payment = findPayment(self.outstanding[-1], self.rate, self.months - self.teaserMonths)
           
        Mortgage.makePayment(self)

In [2]:
 def compareMortgages(amt, years, fixedRate, pts, ptsRate, varRate1, varRate2, varMonths):
        totMonths = years*12
        fixed1 = Fixed(amt, fixedRate, totMonths)
        fixed2 = FixedWithPts(amt, ptsRate, totMonths, pts)
        twoRate = TwoRate(amt, varRate2, totMonths, varRate1, varMonths)
        morts = [fixed1, fixed2, twoRate]
        for m in range(totMonths):
            for mort in morts:
                mort.makePayment()
        for m in morts:
            print(m)
            print(' Total payments = $' + str(int(m.getTotalPaid())))
            print('')

In [4]:
compareMortgages(amt = 200000, 
                 years = 30, 
                 fixedRate = 0.07, 
                 pts = 3.25, 
                 ptsRate = 0.05, 
                 varRate1 = 0.045, 
                 varRate2 = 0.095, 
                 varMonths = 48)

Fixed, 7.0%
 Total payments = $479017

Fixed, 5.0%, 3.25 points
 Total payments = $393011

4.5% for 48 months, then 9.5%
 Total payments = $551444

